# Deep RL hands-on by Maxim Lapan
## Chapter 6 Deep Q-Networks

* conda activate gym
* [source github](https://github.com/PacktPublishing/Deep-Reinforcement-Learning-Hands-On)
* from this part, it need modularized code. Some kind of code woudl be packed in lib directory. Be aware that

## DQN for Pong

## Q-learning algorithm 복습

1. Q(s,a)를 초기화한다.(Q는 action까지 특정한 가치함수임)
2. 환경과 상호작용하여, tuple(s,a,r,s')를 얻는다.
3. 손실값을 계산한다. 
    1. 에피소드가 종료될땐 L = (Q(s,a) - r)^2
    2. 그렇지 않다면 L = (Q(s,a) -(r + gamma * max(Q(s',a'), a' in A))^2
4. Q(s,a)를 갱신하는데, stochastic gradient descent (SGD) algorithm를 사용한다. 이 때 경사하강 방향은 Loss를 최소화하는 방향이다.
5. 수렴할 때까지 2번 단계부터 반복한다.

### epsilon-greedy method
* exploitation vs exploration 문제를 해결하기 위한 가장 단순한 방법.
* hyperparameter epsilon를 설정해두고(보통 p 확률 값으로 정한다), 이 값 이하면 random action을 선택 - exploration. 그 이상이면 policy (Q learning의 경우 best Q)를 따른다 - exploitation.



## DQN에서 고려해야 할 요소들

### SGD optimization
* 손실함수를 최소화하는 방식은 supervised learning의 기본이다. 여기에는 몇가지 전제가 깔려 했는데, 가장 중요한 것은 data의 분표가 i.i.d를 따른다는 것이다. independent and identically distributed. 이는 SGD를 사용할 때 중요한 조건이다. (사실 supervised learning에 쓰이는 data set은 i.i.d를 따를지 몰라도, 현실 세계의 사물은 그렇게 i.i.d하진 않다.)
* RL에서는 당연히 i.i.d를 따르지 않는다.
    * env로부터 들어오는 sample은 independent하지 않다. 
    * data를 batch화 하더라도, independent하지 않고, identical하지도 않는다. episode 의존적이고, 이때 내재된 policy에 의존적이기 때문에 identical하지 않은 것이다. 만약 policy가 random이라면 identical할수도 있겠지만, 우리가 원하는 것은 random policy가 아니다. {Successor representation learning에서는 가치와 상태이전함수를 분리했고, 상태이전함수만을 학습하기 위해서는 random policy를 쓰기도 한다.}
* 이 문제를 해결하기 위해서 고안된 것이 replay buffer다. 의존성을 띄는 최근 sample data가 아니라, 훨씬 과거에 저장된 sample을 이용해 SGD를 적용하는 것이다. 하지만, policy가 update되면 지나치가 과거의 sample은 도움되지 않기 때문에 차츰 밀어낸다. 그래서 buffer라고 불리는 것. {이런 고안 배경을 고려하면 replay buffer를 hippocampal memory에 비유하는 것은 적절하지 않아 보인다. 어떤 net을 최적화를 위해 사용되는 buffer와 imagination이나 planning을 위해 사용되는 memory를 좀 다른 듯하다. 물론 cortex에 있는 neural circuit을 최적화하기 위해서 hippocampal memory를 사용한다면 가능한 얘기지만, 과연 그런가? }

### Correlation between steps
* Q(s,a)가 Q(s',a')를 통해 update될 때 tabular method에서는 크게 문제 없지만, neural net에서 보면 s와 s'는 크게 다르지 않다. 그래서 Q(s,a)를 update하면 Q(s',a')도 update될 수 있다. 이러면 학습이 unstable해진다. 
* 그래서 target network를 따로 만들었다. 이건 원래 network의 copy version으로, 여기서 Q(s,a) update용 Q(s',a')값을 가져온다. 그리고 두 개의 net는 주기적으로만 동기화 시킨다. (step으로 봤을때, 1k or 10k iterations 정도)

### The Markov property
* Markov 환경이라면 전체 환경이 관찰되어야 하지만, atari game을 포함해 대부분의 현실 문제는 그렇지 않다. 이걸 partially observable MDPs(POMDP) 문제라 한다. pong의 경우 한 번에 하나의 frame만 관찰 가능하다. 
* atari game 같은 문제를 풀기 위해서 보통 4개의 frame을 stack으로 처리해 훈련시켜서 POMDP를 약간 극복하려 한다. 
    

In [1]:
from lib import wrappers
from lib import dqn_model

import os
import argparse
import time
import numpy as np
import collections

import torch
import torch.nn as nn
import torch.optim as optim

import tensorflow as tf
import datetime

In [2]:
# hyperparameters

DEFAULT_ENV_NAME = "PongNoFrameskip-v4"
MEAN_REWARD_BOUND = 19.5

GAMMA = 0.99
BATCH_SIZE = 32
REPLAY_SIZE = 10000
LEARNING_RATE = 1e-4
SYNC_TARGET_FRAMES = 1000
REPLAY_START_SIZE = 10000

EPSILON_DECAY_LAST_FRAME = 10**5
EPSILON_START = 1.0
EPSILON_FINAL = 0.02

In [3]:
Experience = collections.namedtuple('Experience', field_names=['state', 'action', 'reward', 'done', 'new_state'])

In [4]:
class ExperienceBuffer:
    def __init__(self, capacity):
        self.buffer = collections.deque(maxlen=capacity)

    def __len__(self):
        return len(self.buffer)

    def append(self, experience):
        self.buffer.append(experience)

    def sample(self, batch_size):
        indices = np.random.choice(len(self.buffer), batch_size, replace=False)
        states, actions, rewards, dones, next_states = zip(*[self.buffer[idx] for idx in indices])
        return np.array(states), np.array(actions), np.array(rewards, dtype=np.float32), \
               np.array(dones, dtype=np.bool), np.array(next_states)

In [5]:
class Agent:
    def __init__(self, env, exp_buffer):
        self.env = env
        self.exp_buffer = exp_buffer
        self._reset()

    def _reset(self):
        self.state = env.reset()
        self.total_reward = 0.0

    def play_step(self, net, epsilon=0.0, device="cpu"):
        done_reward = None

        if np.random.random() < epsilon:
            action = env.action_space.sample()
        else:
            state_a = np.array([self.state], copy=False)
            state_v = torch.tensor(state_a).to(device)
            q_vals_v = net(state_v)
            _, act_v = torch.max(q_vals_v, dim=1)
            action = int(act_v.item())

        # do step in the environment
        new_state, reward, is_done, _ = self.env.step(action)
        self.total_reward += reward

        exp = Experience(self.state, action, reward, is_done, new_state)
        self.exp_buffer.append(exp)
        self.state = new_state
        if is_done:
            done_reward = self.total_reward
            self._reset()
        return done_reward

In [6]:
def calc_loss(batch, net, tgt_net, device="cpu"):
    states, actions, rewards, dones, next_states = batch

    states_v = torch.tensor(states).to(device)
    next_states_v = torch.tensor(next_states).to(device)
    actions_v = torch.tensor(actions).to(device)
    rewards_v = torch.tensor(rewards).to(device)
    done_mask = torch.ByteTensor(dones).to(device)

    state_action_values = net(states_v).gather(1, actions_v.unsqueeze(-1)).squeeze(-1)
    next_state_values = tgt_net(next_states_v).max(1)[0]
    next_state_values[done_mask] = 0.0
    next_state_values = next_state_values.detach()

    expected_state_action_values = next_state_values * GAMMA + rewards_v
    return nn.MSELoss()(state_action_values, expected_state_action_values)

In [7]:
device = torch.device("cuda")
# os.environ["CUDA_VISIBLE_DEVICES"] = '0, 1'
# !pip install gym[atari]
env = wrappers.make_env(DEFAULT_ENV_NAME) # "PongNoFrameskip-v4"

In [8]:
'''
pong env.observation space - (4, 84, 84)
action space - 6
'''
net = dqn_model.DQN(env.observation_space.shape, env.action_space.n).to(device)
tgt_net = dqn_model.DQN(env.observation_space.shape, env.action_space.n).to(device)


print(net)

DQN(
  (conv): Sequential(
    (0): Conv2d(4, 32, kernel_size=(8, 8), stride=(4, 4))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
    (3): ReLU()
    (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (5): ReLU()
  )
  (fc): Sequential(
    (0): Linear(in_features=3136, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=6, bias=True)
  )
)


In [9]:
torch.cuda.get_device_name()

'GeForce GTX 1080'

In [10]:
print(tgt_net)

DQN(
  (conv): Sequential(
    (0): Conv2d(4, 32, kernel_size=(8, 8), stride=(4, 4))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
    (3): ReLU()
    (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (5): ReLU()
  )
  (fc): Sequential(
    (0): Linear(in_features=3136, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=6, bias=True)
  )
)


In [11]:
buffer = ExperienceBuffer(REPLAY_SIZE)
agent = Agent(env, buffer)
epsilon = EPSILON_START

In [12]:
optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)
total_rewards = []
frame_idx = 0
ts_frame = 0
ts = time.time()
best_mean_reward = None

In [13]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S") # it's convenient to record the date with current time
LOGDIR = './tmp/ch6' + DEFAULT_ENV_NAME + '/' + current_time + '/' 
writer = tf.summary.create_file_writer(LOGDIR)

while True:
    frame_idx += 1
    epsilon = max(EPSILON_FINAL, EPSILON_START - frame_idx / EPSILON_DECAY_LAST_FRAME)

    reward = agent.play_step(net, epsilon, device=device)
    if reward is not None:
        total_rewards.append(reward)
        speed = (frame_idx - ts_frame) / (time.time() - ts)
        ts_frame = frame_idx
        ts = time.time()
        mean_reward = np.mean(total_rewards[-100:])
        print("%d: done %d games, mean reward %.3f, eps %.2f, speed %.2f f/s" % (
            frame_idx, len(total_rewards), mean_reward, epsilon,
            speed
        ))
        
        with writer.as_default():    
            tf.summary.scalar("epsilon", epsilon, frame_idx)
            tf.summary.scalar("speed", speed, frame_idx)
            tf.summary.scalar("reward_100", mean_reward, frame_idx)
            tf.summary.scalar("reward", reward, frame_idx)
        
        if best_mean_reward is None or best_mean_reward < mean_reward:
            torch.save(net.state_dict(), DEFAULT_ENV_NAME + "-best.dat")
            if best_mean_reward is not None:
                print("Best mean reward updated %.3f -> %.3f, model saved" % (best_mean_reward, mean_reward))
            best_mean_reward = mean_reward
        if mean_reward > MEAN_REWARD_BOUND:
            print("Solved in %d frames!" % frame_idx)
            break

    if len(buffer) < REPLAY_START_SIZE:
        continue

    if frame_idx % SYNC_TARGET_FRAMES == 0:
        tgt_net.load_state_dict(net.state_dict())

    optimizer.zero_grad()
    batch = buffer.sample(BATCH_SIZE)
    loss_t = calc_loss(batch, net, tgt_net, device = device)
    loss_t.backward()
    optimizer.step()

898: done 1 games, mean reward -20.000, eps 0.99, speed 942.30 f/s
2056: done 2 games, mean reward -19.000, eps 0.98, speed 709.29 f/s
Best mean reward updated -20.000 -> -19.000, model saved
2925: done 3 games, mean reward -19.667, eps 0.97, speed 997.27 f/s
3988: done 4 games, mean reward -19.750, eps 0.96, speed 999.06 f/s
4830: done 5 games, mean reward -20.000, eps 0.95, speed 989.96 f/s
5812: done 6 games, mean reward -20.000, eps 0.94, speed 990.58 f/s
6622: done 7 games, mean reward -20.143, eps 0.93, speed 987.01 f/s
7446: done 8 games, mean reward -20.250, eps 0.93, speed 985.68 f/s
8358: done 9 games, mean reward -20.333, eps 0.92, speed 985.98 f/s
9120: done 10 games, mean reward -20.400, eps 0.91, speed 962.95 f/s
9910: done 11 games, mean reward -20.455, eps 0.90, speed 964.69 f/s
10691: done 12 games, mean reward -20.500, eps 0.89, speed 136.60 f/s
11674: done 13 games, mean reward -20.462, eps 0.88, speed 125.45 f/s
12634: done 14 games, mean reward -20.500, eps 0.87, s

KeyboardInterrupt: 